<a href="https://colab.research.google.com/github/jhk990602/datapractice/blob/main/catboost0215%EC%8B%A4%ED%8C%A8%EC%9E%91.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 9.5 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import FunctionTransformer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import (
    accuracy_score,
    confusion_matrix,
    f1_score,
    precision_score,
    recall_score,
    classification_report
)

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [222]:
train_df = pd.read_csv('/content/drive/MyDrive/lg/train.csv')
test_df=pd.read_csv('/content/drive/MyDrive/lg/submission.csv')

# customer_position - 김나현
- nan값은 그대로 있음. other처리하거나 none로 그냥 두거나 여러가지 시도

In [223]:
mapping = {
 'academic coordinator/ post graduate teacher (accountancy, business studies)/ tgt (ict)'
    :  'academic coordinator/ post graduate teacher (accountancy, business studies)/ tgt (ict)', # 1명

 'academic specialist' : 'academic specialist',  # 2명

 'administrative' : 'admin',  # 5명


 'architect/consultant' : 'architect/consultant',  # 2
 'architecture/consult' : 'architect/consultant',   # 1


 'assistant professor' :  'assistant professor',  # 8 조교수
 'assistant professor of enlish' :  'assistant professor', # 1
 'asst prof.' :  'assistant professor',# 1 조교수


 'associate professor' : 'associate professor', # 1  부교수
 'associate professor in electronics engg' : 'associate professor', # 1 전자공학 분야 부교수


 'prof.' :  'professor',# 2
 'professor' :  'professor',# 1


 'professor of mathematics' : 'professor of mathematics', # 2


 'guest faculty' :'guest faculty', # 1  초빙교수


 'physics faculty' :'physics faculty', # 1


 'neet/ olympiad expert faculty' : 'neet/ olympiad expert faculty', # 1


 'quantitative aptitude faculty' :'quantitative aptitude faculty', # 1


 'educator' : 'educator',# 2
 'education' : 'educator', # 1
 'education professional' : 'educator', # 1

 'senior lecturer' : 'senior lecturer',#1

 'maths lecturer' : 'maths lecturer' ,# 1

 'english trainer for ielts,toefl,pte,gre,sat exams.'
    : 'english trainer', # 1   국제 영어 시험인 IELTS, TOEFL, PTE, GRE, SAT 등의 시험을 준비하는 학생들을 대상으로 영어 교육을 제공

 'chemistry teacher' :  'chemistry teacher', # 1

 'science teacher' :'science teacher', # 2

 'physics teacher' :'physics teacher', # 3

 'pgt physics' : 'pgt physics', # 1 대학교나 고등학교에서 대학원 수준의 물리학을 가르치는 교사

 'math and physics teacher' : 'math and physics teacher',# 2
 'physics and mathematics teacher' :'math and physics teacher' , # 2

 'teacher' :'teacher',# 1
 'teacher/middle school coordinator' : 'teacher',# 1

 'professional trainer' :  'professional trainer', # 1


 'hon dean' :  'hon dean', # 1  honorary dean(명예 교장)


 'associate/analyst' : 'associate/analyst',  # 2597


 'av management' : 'av management', # 2  'Audio-Visual Management(음향 및 시각 관리)'


# 애애애애애매매매매매 애매함 ;
 'business development' : 'business development', # 2

 'business development/sales' :'business development/sales',  # 1

 'sales' : 'sales',#1

 'subsidiary sales (ise)' :'subsidiary sales (ise)',# 1


 'business partner' : 'business partner',  # 2

 'business unit director' : 'business unit director', # 1


 'c-level executive' : 'c-level executive',# 851
 'c-levelexecutive' : 'c-level executive', # 6
 'ceo/founder' : 'c-level executive', # 7984
 'ceo/fundador' : 'c-level executive', # 2
 'chief executive officer' : 'c-level executive', # 1
 'the big boss' :'c-level executive', # 1


 'founder' : 'founder', # 2


 'co-founder' : 'co-founder', # 1


 'career coach' : 'career coach', # 1


 'chairman' :  'chairman', # 1


 'commercial consultant' :  'commercial consultant', # 1

 'consultant' : 'consultant',  # 23
 'consulting' : 'consultant',  # 2


 'commercial end-user' : 'commercial end-user',  # 7

 'end-user' : 'end-user', # 424


 'customer' : 'customer',  # 5


 'decision influencer' : 'decision influencer', # 9
 'decision-influencer' : 'decision influencer', # 7


 'decision maker' : 'decision maker',  # 9
 'decision-maker' : 'decision maker',#3


 'director' : 'director',#4843
 'director cum faculty at gaining apex coaching centre'
    : 'director cum faculty at gaining apex coaching centre', # 1  'Geting Apex 코칭 센터의 교수 겸 감독'


 'distributor': 'distributor', # 1


 'employee' : 'employee', # 14


 'engineering' : 'engineering', # 9


 'entrepreneurship' : 'entrepreneurship', # 3


 'entry level' :'entry level', # 2216
 'entrylevel' : 'entry level', # 3

 'intern' :'intern', # 446

 'exhibition' :  'exhibition', # 3


 'főorvos' : 'főorvos', # 1   주치의   'Chief Physician'

 'hospital' : 'hospital', # 201

 'surgery professional' : 'surgery professional',# 1

 'tierarzt' : 'vet' ,# 1   사의사


 'genel müdür' : 'general manager', # 1   'general manager'


 'product management' : 'product management', # 1


 'gerente' :'manager',#3    manager
 'manager' : 'manager',#8217


 'government' :'government' ,# 29


 'homeowner' : 'homeowner', # 21


 'installer' : 'installer', # 60


 'leadership/executive office/owner' :'leadership/executive office/owner', # 8


 'lider de desarrollo' :'development leader',# 1 development leader


 'manufacturer' : 'manufacturer', # 23

 'medical device manufacturer' :  'medical device manufacturer', # 11


 'market intelligence/research' : 'market intelligence/research' ,#1


 'medical imaging specialist' : 'medical imaging specialist' ,# 3


 'operations' : 'operations', # 3


 'other' : 'other', # 6499
 'others' : 'other',# 964

 'other - please specify - cedia association' : 'other - please specify - cedia association', # 2

 'partner' : 'partner', # 2536


 'pathologist' : 'pathologist', # 1


 'president' : 'president', # 1


 'principal & director' : 'principal & director', # 2


 'principal at oxford integrated pu science college' : 'principal at oxford integrated pu science college', # 2


 'proprietário(a)' : 'owner',# 1   owner

 'radiology professional' :'radiology professional', #1


 'research' : 'research',#1


 'software /solution provider' :  'software /solution provider',# 25


 'system integrator' : 'system integrator',# 21


 'technical' : 'technical',# 1
 'técnico' :'technical', # 1


 'trainee' :'trainee',# 849


 'vice president' :'vice president', #509
 'vicepresident' : 'vice president', # 11
 'vp' : 'vice president' ,# 1


# 뭔 직업
 'bulgaria' : 'bulgaria' ,  # 8
 'cargo' : 'cargo', # 27
 'exhibitiontv' :  'exhibition tv', # 1
 'mindenes' :  'mindenes',# 1
 'no influence' : 'no influence', # 3
 'this is a consume display requirement for home purpose.'
    : 'this is a consume display requirement for home purpose.', # 1
 'unpaid' : 'unpaid', # 17




 'none' : 'nan',# 19680
 'not applicable' : 'nan', # 7
}

In [224]:
train_df['customer_position'] = train_df['customer_position'].map(mapping)
train_df['customer_position'] = train_df['customer_position'].replace('nan', pd.NA)

# customer_job - 김나현
- customer_position과 마찬가지로 nan값 그대로 둠

In [225]:
# 자동화 코드 젭알 ㅠ
mapping = {
 'museum / gallery' :  'museum / gallery',  # 1명


# art, design관련
 '3d/vfx art' :'3d/vfx art',

 'art installation' :'art installation',

 'artist, lead on equipment selection' : 'artist, lead on equipment selection',

 'colorist' : 'colorist',

# art and design
 'művészet_és_design' : 'art and design',   # 3명
 'art and design' : 'art and design',
 'arts and design' : 'art and design',
 'arts_and_design' :  'art and design',
 'arte y diseño' :  'art and design',
 'arte_e_design' :  'art and design',

# art ? design?
 'graphic design' : 'graphic design',  #2명
 'graphic/color art' : 'graphic/color art', # 331명

# 다 designer로 같이 묶어야 하는지 말아야 할지 이야기 해봐야할듯
 'design and provide equipment' : 'design and provide equipment',  # 1명
 'design consultant' : 'design consultant',  # 1먕
 'design/build' : 'design/build',         #1명
 'design/decision maker': 'design/decision maker',    # decision maker라는 직업이 있어서 이미 decider로 통일했음
 'design/install/training/support' : 'design/install/training/support',   #1명
 'designer, creative technologist' : 'designer, creative technologist',    # 1명
 'designer/ project manager' : 'designer/ project manager', # 1명
 'designer/pm/gc' : 'designer/pm/gc',   # 1명 designer 도 하고 project manager도 하고 gereral contractor도 하는듯..
 'designere / budget' : 'designer/budget' ,  # 1명
 'designer, producer': 'designer/producer',  # 1명
 'lead designer' :  'lead designer',  # 1명

 'designer/ engineer' : 'design engineer' ,  # 1명
 'design engineer' : 'design engineer',   # 2명
 'senior design engineer' : 'design engineer',# 1명

 'design and install' : 'design and install',
 'design and installation company' : 'design and install',
 'designer/installer' : 'design and install',

 'design/purchaser' : 'design/purchaser',  # 1명
 'designer purchaser' : 'design/purchaser', # 1명

 'designers' : 'designer' , # 1명
 'design' : 'designer',   # 3명
 'designer' : 'designer',    # 14명

 'kreation und design': 'creation and design', # 1명
 'kreation_und_design' : 'creation and design',   # 16명


# manager 집합
 'operations manager' : 'operations manager',  # 6명
 'ops mgr' :'operations manager',   #1명   "Ops Mgr"은 "Operations Manager"의 약어

 'ordering manager' :  'ordering manager' , # 1명

 'office manager' :'office manager',  # 1명

 'owner / project manager' :'owner / project manager', # 1명

 'purchasing manager' :  'purchasing manager',  # 2명


 '5% of hotel needs' : '5% of hotel needs',

 'hotel manager' : 'hotel manager',             # 1명

 'a/v project manager' : 'av project manager',
 'av project manager'  : 'av project manager',

 'r&d project manager' :  'r&d project manager',# 1명

 'site manager' :'site manager',# 1명

 'studio manager' : 'studio manager',# 1명

 'resource manager' : 'resource manager',# 1명


 'av estimator' : 'av estimator',


 'av tech' : 'av technician',
 'av technician' : 'av technician',


 'account exec/manager' : 'account exec/manager',
 'account management' :'account management',


 'accounting' : 'accounting',
 'accounts payable' : 'accounts payable',
 'asset management' : 'asset management',


 'admin' : 'admin',
 'admin assistant' : 'admin',
 'administración' : 'admin',
 'administration' : 'admin',
 'administrative' : 'admin',
 'administrative assistant' : 'admin',
 'adminisztráció' : 'admin',
 'amministrativo' : 'admin',


 'imaging administrator' : 'imaging administrator' ,


 'it admin' : 'it administrator',    # 1명
 'it administrator' : 'it administrator',   # 1명

 'network administrator' :  'network administrator',

 'platform administrator' :  'platform administrator',   # 1명

 'pacs administrator' :  'pacs administrator', # 2명  의료 it 관리자

 'systems administrator' : 'systems administrator',# 1명

 'department secretary' : 'department secretary',

# 마케팅 관련
 'advertising' : 'advertising/marketing' ,
 'advertising and promotions team' : 'advertising/marketing',
 'field marketing' : 'advertising/marketing',
 'event marketing' : 'advertising/marketing',                  # 3명
 'marketing' :'advertising/marketing',      # 1161 명

 'marketing coordinator' : 'marketing coordinator',   # 1명

 'marketing operations' : 'marketing operations',  # 2명

 'product marketing' : 'product marketing',  # 3명


 'appliance specialist' : 'appliance specialist',      # 가전제품 전문가


 'application development' : 'application development', # 응용프로그램 개발


 'architect' : 'architect',
 'architect ass interiores' : 'architect ass interiores',
 'architect/owner' : 'architect/owner',
 'arquitecto/consultor' : 'arquitecto/consultor',
# 'architect ass interiores'랑 묶어야 하나?
 'interior designer' :'interior designer',               # 2명
 'interior stylist' : 'interior stylist',                 # 1명



 'associate/analyst' : 'associate/analyst',


 'authorize (you are responsible for making the final decision)' : 'authorize',  #승인  1명에 대한 정보

# 일단 나눠놓긴 했는데 bidder이랑 합칠것인가
 'bidder' : 'bidder',    # 명령  1명에 대한 정보
 'public bidder' : 'public bidder',# 1명


 'broadcasting & media' : 'broadcasting & media',


 'business development' : 'business development',
 'business_development' : 'business development',

# owner로 합칠것인가? 애매해서 둔 그룹
 'building owner' : 'building owner',
 'property owner' : 'property owner',# 1명 부동산 소유주

 'business owner' : 'business owner',
 'owner representation' : 'owner representation', # 1명
 'owning company' : 'owning company', # 1명

 'owner' :'owner',  # 57명
 'proprietário(a)' : 'owner', # 1명   owner 소유자


 'product owner' : 'product owner',   # 1명


 'buyer' : 'buyer',
 'buyer, coordinating': 'buyer',


 'c-level executive' : 'c-level executive',   # chief level 임원. 1명 정보
 'ceo' : 'c-level executive',   # ceo, ceo/founder다 e-level executive에 속한다고 볼 수 있음
 'ceo/founder' : 'c-level executive',
 'executive' : 'c-level executive',                        # 14명
 'the big boss' : 'c-level executive',# 1명

 'underboss' : 'underboss',# 1명

 'engagement executive' : 'engagement executive',

 'engineering & technical executive' : 'engineering & technical executive',   # 3명

 'finance executive' : 'finance executive',    # 1명

 'marketing executive' : 'marketing executive',  # 2명

 'operations executive' :  'operations executive',   # 2명


 'leader' : 'leader',          # 2명
 'lead' : 'leader',   # 16명


 'deputy cio' : 'deputy cio',


 'coo' : 'coo',    # chief operating officer   최고 운영 책임자


 'chief' : 'chief',

# 기술적 책임자
 'chief eng.' : 'chief engineer',
 'chief engineer' : 'chief engineer',
 'chief of engineering' : 'chief engineer',
# 기술적 주도 및 지도
 'lead engineer' : 'lead engineer',   # 1명


 'principal engineer' : 'principal engineer', # 1명


 'car dealership' : 'car dealership',


 'cctv monetoring' : 'cctv monetoring',
 'cctv view' : 'cctv monetoring',


 'chef' : 'chef',


## doctor과 surgeon을 같은 직업으로 넣을 것인가? clinic은 어떻게 할것인가
 'chirurgien' : 'surgeon',        # 1명  외과 의사
 'cirugano' : 'surgeon',          # 1명  외과의사인듯 스페인어로 cirujano가 외과의사.한 사람에 대한 정보인데 그 사람 국적이 스페인

 'doctor' : 'doctor',             # 1명   business area 'hotel & accommodation'
 'főorvos' : 'doctor',             # 1명   헝가리어로 주치의

 'profesional de cirugía' : 'surgery professional',  #3명 'surgical professional' 수술 전문가
 'surgery professional' : 'surgery professional',# 118명
 'surgery professional\u200b' : 'surgery professional',#6명

 'profesional de radiología' : 'profesional de radiología',  #1명  'radiology professional' 방사선 전문가

 'tierarzt' : 'tierarzt',# 1명  수의사

 'clinic' : 'clinic',            # business area가 hospital & health care임 직업이 외과의사인 것들 business area가 nan이던데 채워도 되지 않을까..?

 'clinical specialist' : 'clinical specialist',  # business area 종류가 {'education','hospital & health care', 'hotel & accommodation', nan, 'public facility'}


 'cloud / mobility' : 'cloud / mobility',  # 1명


 'collaboration & web apps': 'collaboration & web apps', # 2명


 'comanager' : 'comanager', # 1명


 'commander' : 'commander',  # 사령관 1명


 'community and social services' : 'community and social services',
 'community_and_social_services' : 'community and social services',


 'community theater' : 'community theater',     #1명


 'home theater' : 'home theater',             # 1명


# 통일 해 말어
 'component of video wall' : 'component of video wall', # 1명
 'part of video wall' :'part of video wall',  # 1명
 'video wall' : 'video wall',# 1명
 'videowall' : 'video wall',# 1명

# 'computing & it'로 합침.....(나머지도 computing&it로 합쳐도 될지 논의)
 'computing & it' : 'computing & it', # 5명
 'it' : 'computing & it',    # 4명
 'it - information technology' : 'computing & it',  # 3명

 'it department' :  'it department', # 1명
 'it hardware technician' : 'it hardware technician',   # 1명
 'it integrator' :  'it integrator',     # 1명
 'it manager' : 'it manager',       # 6명
 'it project lead' :  'it project lead',   # 1명
 'it specialist' :  'it specialist',   # 2명
 'it support' :  'it support',    # 1명
 'it tech.' : 'it tech.',         # 1명
 'it/software' : 'it tech.',      # 2명

 'office it' :  'office it',   # 1명

 'software developer' : 'software developer',# 1명

 'software solution' :  'software solution',# 1명


 'construction manager' : 'construction manager',    # 1명


## consultant로 묶을것인가
 'consultant' : 'consultant',
 'consultent' : 'consultant',
 'consulting' : 'consultant',

 'consultant / purchaser' : 'consultant / purchaser',    # 1명인데 consultant로 넣을지 말지
 'consultant,cabinet fabricator': 'consultant,cabinet fabricator', # 마찬가지로 1명인데 consultant라고 할지 말지
 'content creation, eq consultant' : 'content creation, eq consultant',  # 마찬가지로 1명인데~


# contractor로 묶을 것인가
 'contractor' : 'contractor',
 'contractor/owner' : 'contractor/owner',    # 1명인데 contractor로 묶을지

 'federal government contractor' : 'federal government contractor',      # 1명

 'sub contractor' :  'sub contractor',# 1명


 'contributor' :  'contributor',


 'coordinator' : 'coordinator'  ,

 'parts coordinator' :  'parts coordinator',   # 1명


 'corporate / office' : 'corporate / office',


 'correspondence' : 'correspondence',


 'curation' : 'curation',


 'customer experience' : 'customer experience',


 'decider' : 'decider',
 'decision maker' : 'decider',


 'developer' : 'developer',

 'developer/property' : 'developer/property',

 'development coordinator/procurement' : 'development coordinator/procurement',


# director로 묶을것인가?
 'director' : 'director', # 17명
 'overseer' : 'director', # 1명
 'supervisor' : 'director',# 2명

 'director comercial' : 'director comercial',      # 1명
 'creative director' :  'director comercial',  #  1명 크리에이티브 디렉터(광고제작 총책임자)

 'director it' : 'director of it',                  # 1명
 'director of it' : 'director of it',               # 3명
 'director,it' : 'director of it',                  # 1명
 "i'm directing it" : 'director of it',              # 1명
 'it director' : 'director of it',   # 5명
 'it dairector' : 'director of it',   # 1명    it director이 오타난거라고 생각하겠음..

 'director of engineering' : 'director of engineering',           # 1명
 'engineering director' : 'director of engineering',     #1명

 'director of finance' : 'director of finance',               # 1명

 'director of lodging' : 'director of lodging',               # 1명

 'director of operations' : 'director of operations',            # 1명
 'regional director of operations' : 'regional director of operations',# 1명 지역 운영 총괄

 'director purchaser' : 'director purchaser',                # 1명

 'directeur technique' : 'director technique',               # 1명

 'f&b director for bicycle casino' :  'f&b director for bicycle casino' ,   # 1명

 'ownner-marketing director' : 'ownner-marketing director',  # 1명

 'program directors' : 'program directors',  # 1명

 'project director' : 'project director', # 1명

 'purchasing director' : 'purchasing director',# 1명


## digital 분야로 합칠 것인가
 'digital display vs signage need' : 'digital display vs signage need',   # 1명
 'digital project manager' : 'digital project manager',           # 1명
 'digital signage' : 'digital signage',                   # 1명


## 전부 display로 통일할 것인가? (일단 display로 통일하긴 했음)
 'display' : 'display',                           # 1명
 'display our products' : 'display',              # 1명
 'display screen' : 'display',                    # 1명
 'display screen from control' : 'display',       # 1명

 'using for window display' :  'using for window display',# 1명

 'restaurant display' : 'restaurant display',# 1명

# distributor 로 통일함
 'distribuidor': 'distributor',                      # 2명
 'distributor' : 'distributor',                       # 3명
 'distributor quotation' : 'distributor',             # 1명


 'drop, purchase maxhub' : 'drop, purchase maxhub',             # 1명    maxhub가 스마트 화상회의 솔루션 제공회사.  회의실 기술을 구현하거나 업무 환경을 개선하는 데 관련된 직업인듯


# education이 더 넓은 폭이라고 생각되어 education으로 통일함
# (higher education과 'institute & academy'은 education으로 합칠것인가)
 'education' : 'education',                         # 2278명
 'educator' : 'education',                          # 3명

 'higher education (college & university)' :  'higher education (college & university)',  #3명

 'institute & academy' : 'institute & academy',               # 3명

 'instructor' : 'instructor',                       # 1명 강사



 'electrical contractor' : 'electrical contractor' ,      # 1명   전기 시설물 관련 작업


 'electronics & telco' : 'electronics & telco',        # 2명  전자제품 및 통신 기술 관련 업무


 'electronics evaluator' : 'electronics evaluator',      # 1명   전자제품이나 장치 성능 평가


 'elevator company' : 'elevator company',         # 1명


 'emerging technology / innovation' : 'emerging technology / innovation',    # 1명


 'energy' : 'energy',         # 1명


# engineering로 통일함
 'engineer' : 'engineering',        # 11명
 'engineering' : 'engineering',     # 6342 명
 'engineering & technical' : 'engineering',   # 13명

 'engineering, design, and install' : 'engineering, design, and install',   #1명
   # 이걸 engineering에 포함시켜야할지


 'enterprise resource planning' : 'enterprise resource planning' ,   # 1명


 'entrepreneurship' : 'entrepreneurship',               # 1192명


# 묶을지 말지 애매
 'entry level' :' entry level',                     # 1명  신입사원을 의미하는듯

 'intern' : 'intern',                           # 1명


# equipment로 묶을지 말지
 'equipment and app provider' : 'equipment and app provider',      #1명
 'equipment custodian' : 'equipment custodian',             # 1명
 'equipment planner' : 'equipment planner',               # 1명
 'equipment selection' : 'equipment selection',             # 1명


 'estimator' : 'estimator',                        # 2명 - 주로 건설 및 제조 업계에서 사용되는 직업으로, 프로젝트나 제품의 비용을 예측하고 계산하는 역할


 'exhibition / convention center' : 'exhibition / convention center' ,    # 1명


# general manager
 'genel müdür' :  'general manager' ,         # 1명
 'genera manager': 'general manager',      # 1명
 'general manager' : 'general manager',     # 11명
 'general manager (decision maker)' : 'general manager',   # 1명
 'general manager - project manager' : 'general manager',   # 1명
 'general manager- purchaser' : 'general manager',         # 1명
 'general management' : 'general manager', # 27명
 'gm' : 'general manager',                                  # 10명
 'gm/part owner' : 'general manager',
 'gestión_de_proyectos' : 'general manager',                # 1명   프로젝트 매니저
 'reseorot general manager' : 'general manager', # 1명

 'vp/gm' : 'vp/gm',# 1명  "Vice President/General Manager" -> 둘 중 어디로 분류해야할지 애매

 'vice president' : 'vice president',# 3명   부사장


# hardware로 묶을지   hardware design engineer은 design?
 'hardware' : 'hardware',           # 1명
 'hardware design engineer' :'hardware design engineer',  #1명
 'hardware selection' :'hardware selection',        # 1명


# facilitator로 묶음
 'facilitator' : 'facilitator',                       # 1명
 'facilitator installation services' : 'facilitator',  # 1명
 'facilities' : 'facilitator',                         # 1명
 'facilities and operations' : 'facilitator',          # 1명
 'facility administrator' : 'facilitator',             # 1명
 'facility manager' : 'facilitator',                   # 1명


# finance로 묶음
 'finance' : 'finance'  ,             # 463명
 'finanzas' : 'finance',              # 1명
 'finanzen' : 'finance',              # 10명
 'pénzügy' : 'finance',  # 4명  finance 재정


# 'human resources' 로 묶음
 'hr' : 'human resources',                         # 2명
 'hr posting' : 'human resources',                  # 1명
 'human resources' : 'human resources',              # 111명
 'human_resources' : 'human resources',               # 1명


# general contractor로 묶음
 'gc' : 'general contractor',                  # 1명
 'general contractor' :'general contractor',  # 1명


# information technology로 묶음
 'informatics, touch capability' : 'information technology',   # 1명
 'information technology' : 'information technology',           # 1937명
 'information technology\u200b' : 'information technology',    # 4명
 'information_technology' : 'information technology',         # 8명


 'influencer' : 'influencer',                     # 2명


 'infrastructure' : 'infrastructure',                 # 1명


 'inquiry-to-buy/contact-us test' :'inquiry-to-buy/contact-us test',  # 1명


# installer로 묶?
 'installer' :  'installer',                       # 8명
 'installer.': 'installer',                   # 1명

 'installer/ system integrater' : 'installer/ system integrater',   # 1명

 'installer/sales rep' : 'installer/sales rep',              #1명

 'installation and purchaser' :  'installation and purchaser',      # 1명


 'obtain quotes, process purchase' : 'obtain quotes, process purchase',   # 1명


 'system designer, integrator' : 'system designer, integrator',# 1명
 'system engineer' : 'system engineer',# 1명
 'system installer' : 'system installer',# 1명
 'systems engineer' : 'systems engineer',# 1명

 'systems design' :  'systems designer',# 1명
 'systems designer' : 'systems designer',# 1명


# integrator로 묶음
 'integrador' : 'integrator',                      # 6명
 'integration' : 'integrator',                     # 1명
 'integrator' : 'integrator',                      # 6명
 'intergrator' : 'integrator',                    # 1명


 'investigator' :  'investigator',            # 1명


# 'manufacturing'
 'manufacturer' : 'manufacturing' , # 1명
 'manufacturing factory / plant' : 'manufacturing',   # 1명


# 'leader'
 'leader' : 'leader',          # 2명
 'lead' : 'leader',   # 16명


 'legal' : 'legal',   # 155명


# maintenance로 통일할것인가
 'maintenance' :  'maintenance',     #1명
 'maintenance supervisor' : 'maintenance supervisor',   # 1명
 'maintenance technician' : 'maintenance technician',  # 1명


# management로 통일?
 'management' : 'management',   # 2명

 'manager' : 'manager',    # 53명
 'managgere' : 'manager',  # 1명

 'managing contractor' : 'managing contractor',   # 1명
 'managing director' :  'managing director',      # 1명
 'managing employee' :  'managing employee',     # 1명
 'managing partner' :'managing partner',     # 1명



#  'media and communication'
 'media and communication' :  'media and communication',   # 709명
 'media and communications' :  'media and communication',   # 1명
 'media_and_communication' :  'media and communication',    # 1명
 'media_e_comunicazione' :  'media and communication',     # 5명
 'medien_und_kommunikation':  'media and communication',   # 11명
 'medios_de_comunicación' : 'media and communication',    # 2명
 'média_és_kommunikáció' :'media and communication',  # 3명
 'média_és_kommunikáció' :'media and communication',  # 3명

# 'medical imaging specialis'
 'medical imaging  specialist' : 'medical imaging specialist',  # 3명
 'medical imaging specialist' : 'medical imaging specialist',   # 198명
 'spécialiste_en_imagerie_médicale' : 'medical imaging specialist',# 1명  expert_in_medical_imaging


# 'military and protective services'
 'military and protective services' :'military and protective services',     #85명
 'military_and_protective_services' :'military and protective services',     # 1명


# 'medical solution provider'
 'medical solution  provider' :'medical solution provider',   # 5명
 'medical solution provider' : 'medical solution provider',    # 148명
 'medical solution provider\u200b' : 'medical solution provider',  #2명


# 'sourcing / procurement'
 'sourcing' : 'sourcing / procurement',# 1명
 'sourcing & quoting for end user' : 'sourcing / procurement',# 1명
 'sourcing / procurement' :  'sourcing / procurement',# 1명
 'sourcing/procurement' :  'sourcing / procurement',# 1명


# 무슨 직업인지는 모르겠으나 end user로 묶음
 'cliente final' :'end user',     # 스페인어 번역 결과 최종 고객/ 뭔 직업이누 / 두 사람에 대한 정보인데 스페인어를 쓰는 멕시코와 콜롬비아가 가 국적
 'end user' : 'end user',         # 2명
 'main end user of the product' : 'end user',    # 1명
 'primary end-user' :'end user',  # 1명

 'user' : 'user', #2명


 'execution' : 'execution',                        # 1명


 'fashion' : 'fashion',                            # 2명


 'field / outside sales' : 'field / outside sales',   # 1명


 'film production' : 'film production',     # 241명


 'final approval' : 'final approval',      # 2명


 'fixing tv' : 'fixing tv',           # 1명

 'change tv' : 'change tv',


 'furnish and install' : 'furnish and install', # 1명


 'global lead of production' : 'global lead of production',            # 1명


 'head' :'head',                       # 1명

 'head of technology' : 'head of technology',          # 1명


 'healthcare professionals' : 'healthcare professionals',   # 3명

 'healthcare services' : 'healthcare services',       # 632명
 'healthcare_services' : 'healthcare services',         # 4명


 'mental health' : 'mental health'  ,     # 1명


# 'operations'
 'operaciones' :'operations',   # 1명
 'operations' : 'operations',   # 1961명
 'üzemeltetés' : 'operations', #2명     헝가리어에서 "운영" 또는 "운영 관리"를 의미

# 같은 직업으로 합쳐도 되는가 (일단 합침...chatgpt한테 물어봄 ㅠ)
 'help desk / desktop services' : 'help desk / desktop services',   # 1명
 'helpdesk specialist' : 'help desk / desktop services',            #1명


#'president'
 'president' :'president',     # 2명
 'president for sennco' :'president',   # 1명


# 'procurement'
 'procurement' : 'procurement',  # 7명
 'procurement specialist' :'procurement',  # 2명
 'procurment' : 'procurement',  #1명    procurement 오타인듯


# 'product researcher'
 'product research' :'product researcher',   # 1명
 'product researcher' :'product researcher',   #1명


# 'product management'
 'product_management' :'product management',  #1명
 'product management' :'product management',   # 573명


# 'program and project management'
 'program-_és_projektmenedzsment' :'program and project management',  # 2명
 'program_and_project_management' : 'program and project management', # 8명
 'program_and_project_manager' : 'program and project management',    #3명
 'programm- und projektmanagement' : 'program and project management',   # 1명
 'programm-_und_projektmanagement' : 'program and project management',   # 13명
 'program and project management' : 'program and project management',  # 948명
 'projektmenedzsment\tprogram and project management' : 'program and project management', # 5명    program and project management


# project manager
 'project manage' : 'project manager',  # 1명
 'project manager' : 'project manager', # 46명
 'projectr mgmt' : 'project manager',# 1명   project management의 오타로 추정됨
 'pm' : 'project manager',    # 4명
 'projection manager' : 'project manager',# 1명

 'project manager / estimator' : 'project manager / estimator', # 1명

 'project manager / principal' : 'project manager / principal',  # 1명
 'principal' : 'principal',   # 1명

 'project manager/designer' : 'project manager/designer',  # 1명

 'producer/project manager' : 'producer/project manager',  # 1명

 'producer' : 'producer',  # 2명

 'project researcher' : 'project researcher',# 1명

 'project sales/manage' : 'project sales/manage',# 1명

 'project team member' : 'project team member',# 1명


 'project administrator' : 'project administrator',  # 1명
 'project architect' : 'project architect',   # 2명
 'project coordinator' : 'project coordinator',  # 5명
 'project designer' : 'project designer',  # 1명
 'project engineer' : 'project engineer', # 1명
 'project facilitator' : 'project facilitator',  # 1명


# project head
 'project head' : 'project head',  # 2명
 'project lead' : 'project head',  # 3명


# purchase
 'purchase' : 'purchase',  # 3명
 'purchase dept' : 'purchase', # 1명
 'purchaser' : 'purchase', # 15명
 'purchasers' : 'purchase',# 1명
 'purchasing' : 'purchase',  # 1938명
 'purchasing agent' :'purchase',  # 3명
 'purchasing authority' :'purchase', # 1명
 'purchasing coordinator' :'purchase',# 1명
 'purchasing supervisor' : 'purchase',# 1명
 'purchsing' : 'purchase',# 1명
# install이 있어서 일단 따로 분류했음..
 'purchaser, it and installer' : 'purchaser, it and installer', # 1명
 'purchase and install' : 'purchase and install', # 1명


 'planner/purchaser' : 'planner/purchaser',   # 1명

 'planner' : 'planner',    # 2명

 'planning and installation' : 'planning and installation', #1명


#'quality assurance'
 'quality assurance' : 'quality assurance',# 39명
 'quality_assurance' : 'quality assurance', # 4명


# 'radiology professional'
 'radiology  professional': 'radiology professional', # 3명
 'radiology professional' : 'radiology professional', # 207명
 'radiology_professional' : 'radiology professional', # 1명


# recommend로 통합함
 'recommend' : 'recommend', #1명
 'recommend (you recommend specific products or technologies for the solution)' : 'recommend',  # 2명
 'recommendation': 'recommend',# 1명
 'recommender': 'recommend',# 1명


 'sign company' : 'sign company',# 1명

 'signage for an attraction' : 'signage for an attraction',# 1명

 'signage manager' : 'signage manager',# 1명

 'signage subcontractor p/m' : 'signage subcontractor p/m',# 1명


# replacing tv
 'replacement tv' : 'replacing tv',# 1명
 'replacing tv' : 'replacing tv',# 1명


 'research' : 'research',  # 134명


 'research & development' : 'research and developement', # 7명
 'research and developement' : 'research and developement',# 1명


 'research and instalaltion' : 'research and instalaltion',# 1명
 'research/install' : 'research and instalaltion',# 1명


 'research products and prices' : 'research products and prices',# 1명


 'sale' : 'sales',# 1명
 'sales': 'sales',# 2218명
 'sales rep' : 'sales',# 1명
 'salesman' : 'sales',# 1명
 'vendite' :  'sales',#5명   판매 sales
 'vertrieb' : 'sales',# 12명    판매 sales
 'értékesítés' : 'sales',# 2명   헝가리어로 "판매"를 의미

 'sales operations' : 'sales operations', # 3명
 'sales engineering' : 'sales engineering', #4명
 'sales executive' : 'sales executive',# 3명
 'sales manager' : 'sales manager', # 4명


# serving
 'serving' : 'serving',# 1명
 'serving food' : 'serving',# 1명
 'serving robot' : 'serving',# 1명


 'tester' :'tester',# 1명


 'testing and troubleshooting' : 'testing and troubleshooting',# 1명


 'the person with the credit card' : 'the person with the credit card',# 1명


 'tradeshow event' :  'tradeshow event',# 1명


 'tv studio manager' :  'tv studio manager',# 1명


 'vendor / reseller' : 'vendor / reseller',# 1명  공급업체/리셀러


 'supplier' :  'supplier',# 1명

 'supplier and installation' : 'supplier and installation',# 1명


 'waiter' : 'waiter',# 1명


# 'teacher'
 'teacher' : 'teacher',# 1명
 'teaching': 'teacher',# 1명


#  'team leader'
 'team lead' :  'team leader',# 1명
 'team leader' :  'team leader',# 1명


 'solution advisor' : 'solution advisor',# 1명

 'solution engineer' : 'solution engineer',# 1명

 'solution provider' : 'solution provider',#2명

 'solutions architect' : 'solutions architect',# 1명

 'solutions provider and specifier' :'solutions provider and specifier'  ,# 1명


 'pathologist' : 'pathologist',  # 48명 병리학자


 'photographer' : 'photographer',   #1명
 'photos' : 'photographer',       # 1명


# technical
 'technical' : 'technical', # 18명
 'tech' : 'technical',#2명
# 그냥 따로 분류함
 'tech service' : 'tech service',# 1명
 'technical / decision maker' :'technical / decision maker',# 1명
 'technical advisor, reseller' :'technical advisor, reseller',# 1명
 'technical director' : 'technical director', # 2명
 'technical marketing': 'technical marketing',# 2명
 'technical sales' :'technical sales',# 1명
 'technology consultant' : 'technology consultant',# 1명
 'technology designer' : 'technology designer',# 1명



 'organizer' : 'organizer',   # 1명


 'post install support and service' :  'post install support and service',  # 1명


 'principal in charge' : 'principal in charge',  #1명


 'quotation curator' : 'quotation curator',# 1명

 'quote gathering/proposer to owner' : 'quote gathering/proposer to owner',# 1명

 'quoting project' : 'quoting project',# 1명


 'railway & metro station' : 'railway & metro station', # 2명


 'real estate' : 'real estate', # 295명


 'renewable energy' : 'renewable energy',# 1명


 'repair uhd 120 hz units' : 'repair uhd 120 hz units',# 1명


 'requirement close' : 'requirement close',# 1명 ?

 'requirements and buyer' : 'requirements and buyer',# 1명


 'requisition' : 'requisition',# 1명


 'reseller' : 'reseller',  # 4명

 'reseller/integrator' : 'reseller/integrator',# 1명


 'retailer/installer' : 'retailer/installer',# 1명


 'revendedor' : 'revendedor', # 7명


 'reviewer' : 'reviewer',# 1명


 'seller installer' : 'seller installer',# 1명


 'service coordinator' : 'service coordinator',# 1명


 'sho lyrics' : 'sho lyrics',# 1명  음악 가사 작성하는 직업으로 추정됨


 'sliding pictures of beauty salon' :  'sliding pictures of beauty salon',# 1명


 'sme' : 'sme',# 1명  중소기업의 약어


 'specifier / integrator' : 'specifier / integrator',# 1명


 'stakeholder' : 'stakeholder',# 1명


 'strategic communications':'strategic communications',# 1명


 'strategy & operations specialist': 'strategy & operations specialist',# 1명


 'submitting proposal' :  'submitting proposal',# 1명


 'support' : 'support',#358명
 'support/facilitator, designer' : 'support/facilitator, designer',# 1명


 'store promotions' : 'store promotions',# 1명



# 뭔 직업인지 모르겠는 그룹
 'altro' : 'altro', # 회사이름인 거 같기도 한데 뭔 직업인지 모르겠
 'asking for quote for client' : 'asking for quote for client',
 'assist in serving food' : 'assist in serving food',      # 1명에 대한 데이터. 다른 그룹에 넣을 수 있으면 넣는게 좋을까
 'cintractor'  : 'cintractor',         # 무슨 직업인지 모르겠음 contractor인듯. 한사람에 대한 정보인데 국적이 미국
 'conference room' : 'conference room', # 1명
 'conference table' : 'conference table', # 1명
 'costar av team' : 'costar av team',
 'details send' : 'details send',   # 2명
 'following up' :'following up',        # 2명
 'for confrence' :'for confrence',       # 1명
 'for presentations' : 'for presentations',   # 1명
 'guestroom tv' :'guestroom tv' ,      # 1명
 'hotel tv' : 'hotel tv',                   # 1명
 # 아래 2개 need 1 tv로 통일해야할까
 'need 1 tv 55" edge led 4k uhd' : 'need 1 tv 55" edge led 4k uhd',    # 1명
 'need one tv' : 'need one tv',          # 1명
 'implement' : 'implement',                      # 2명
 'liason' : 'liason',     # 1명
 'manger' :'manger', # 1명
 'master mind' :  'master mind',     # 1명
 'menu' : 'menu',     # 1명
 'mindenes' : 'mindenes' ,  # 1명 헝가리어로 모든 것
 'si' : 'si',# 1명
 'no requirment' : 'no requirment',    # 1명
 'no respoxse on phone will try again' :'no respoxse on phone will try again',   #1명
 'nothing' : 'nothing',   # 1명
 'test4' :  'test4',# 1명
 'wall mounted screen mirroring' : 'wall mounted screen mirroring',# 1명
 'we are in iceland' : 'we are in iceland',# 1명
 'var' : 'var' ,# 1명
 'other stores' : 'other stores',  # 3명
 'ranger 2' : 'ranger 2',   # 1명
 'pricing' : 'pricing',   # 1명
 'primary' : 'primary',  #1명



# other로 통일한 그룹
 'egyéb' : 'other',                             # 5명  헝가리어로 else/ other의 뜻인 듯
 'autres' : 'other', #  2명    프랑스어로 other의 뜻인듯
 'other' :'other',  # 4852명
 'others' :'other', # 763명
 'otro':'other', # 8명
 'otros':'other', # 1명
 'sonstiges' : 'other',# 10명  독일어로 "기타"나 "다른 것"을 의미

# n.a는 개인적으로 nan값이라고 봐야할 것 같다
############ nan이라는 string으로 매핑해두고 나중에 nan이라는 string을 nan값 처리해줘야함 ###########
    'n.a' : 'nan'      # 1명
#  nan,  # 18733


}

In [226]:
train_df['customer_job'] = train_df['customer_job'].map(mapping)
train_df['customer_job'] = train_df['customer_job'].replace('nan', pd.NA)

# inquiry_type - 김나현
- 급하게 정말 같은 글자인데 다르게 표기된 것만 매핑

In [227]:
mapping  = {

 'ETC.' :  'ETC.',
 'Etc.' :  'ETC.',



 'Quotation or Purchase Consultation' :  'Quotation or purchase consultation',
 'Quotation or Purchase consultation' :  'Quotation or purchase consultation',
 'Quotation or purchase consultation' :  'Quotation or purchase consultation',


 'Sales Inquiry' :  'Sales Inquiry' ,
 'Sales inquiry' : 'Sales Inquiry' ,





 'other' : 'other',
 'other_' : 'other',
 'others' : 'other',
 'Other' : 'other',
 'Others' :'other',


 'Usage or Technical Consultation' :  'usage or technical consultation',
 'Usage or technical consultation' :   'usage or technical consultation',
 'usage or technical consultation' :  'usage or technical consultation',
 'usage_or_technical_consultation' :  'usage or technical consultation',
}

In [228]:
train_df['inquiry_type'] = train_df['inquiry_type'].replace(mapping)

# customer_country - 김주호

In [229]:
# / 슬래시 2개 뒤에 보통 국가이름, 그냥 국가만 적혀있거나 메일형태 or . 으로 나뉜경우도 있
#df['연도']=list(map(lambda x : int(x.split(".")[0].split()[0].split("/")[0]), df["기간"]))
train_df['customer_country']=train_df['customer_country'].apply(lambda x : str(x).split('/')[-1])

In [230]:
#@있는거빼기
train_df.loc[train_df['customer_country'].str.contains('@', na=False), 'customer_country'] = np.nan
#list로정리
India_list = ['Anand Vihar Delhi', 'Botswana', 'Chennai', 'Pune', 'Telangana', 'bangalore', 'bhilwara', 'gujarat', 'gurgaon', 'hyderabad', 'indore', 'kerela', 'mumbai', 'odisha', 'uttar pradesh']
Spain_list=['ALICANTE','CACERES','GRAN CANARIAS PLAYA DEL INGLES','MADRID','VALENCIA','spain']
Brazil_list=['Brazil','Aparecida','Belo Horizonte','Capão da canoa','Centro de Produção Audiovisual - Sesc São Paulo','Cuiabá','Dourados','Foz de Iguaçu - PR Avenida Tancredo Neves 6731 Jardim Itaipu','João Pessoa','Manaus','Recife','Rio de Janeiro','Sao Paulo','São Paulo']
Colombia_list=['Barranquilla','Bucaramanga','BOGOTA','Cartagena','colombia']
Vietnam_list=['Ha Noi','vietnam']
Turkey_list=['Türkiye','Turkey']
UAE_list=['UAE Dubai']#U.A.E로 넣어야함
Greece_list=['Greece','Θέση Πέτσα Βακαλοπούλου ΒΙΟΠΑ Παλλήνης  153 51']
Italy_list=['italy','90044 Carini']
Netherlands_list=['Netherlands']
CotedIvoire_list=['Ivory Coast']#Cote d'Ivoire 로 등록하자
Saint_Kitts_list=['St Kitts'] #Saint Kitts and Nevis로 등록하자
Puerto_Rico_list=['Puerto Rico'] #Puerto Rico로 등록하자
USA_list=['US','United States','Jacksonville Florida','New Hampshire','Nevada','Ohio','Zip 98433']#United States로 등록하자
Peru_list=['PerU']
Dominica_list=['Dominican republic']

country_lists = [India_list,Spain_list, Brazil_list, Colombia_list, Vietnam_list, Turkey_list, UAE_list, Greece_list,Italy_list,Netherlands_list,CotedIvoire_list,Saint_Kitts_list,Puerto_Rico_list,USA_list,Peru_list,Dominica_list]
country_names = ['India','Spain', 'Brazil', 'Colombia', 'Vietnam', 'Turkey', 'U.A.E', 'Greece','Italy','Netherlands',"Cote d'Ivoire",'Saint Kitts and Nevis','Puerto Rico','United States','Peru','Dominican Republic']

for i in range(len(country_lists)):
    for city in country_lists[i]:
        train_df.loc[train_df['customer_country'].str.contains(city, case=False, na=False), 'customer_country'] = country_names[i]

In [231]:
#숫자들어간것 다 미국으로 바꾸기
train_df.loc[train_df['customer_country'].str.contains('\d', regex=True, na=False), 'customer_country'] = 'United States'
#맨앞글자가 공백이면 공백없에기?
train_df['customer_country'] = train_df['customer_country'].str.lstrip()

In [232]:
replace_values = {'A': 'United States', 'BR': 'United States', 'CA': 'United States','ND': 'United States','NY': 'United States','RJ': 'United States'}
train_df['customer_country'] = train_df['customer_country'].replace(replace_values)
replace_value2 = {'country': np.nan, 'lucknow': np.nan,'': np.nan}
train_df['customer_country'] = train_df['customer_country'].replace(replace_value2)
mode_country = train_df.groupby('response_corporate')['customer_country'].agg(pd.Series.mode).to_dict()

# 'customer_country'열의 결측값을 해당 회사의 최빈 국가로 채웁니다.
train_df['customer_country'] = train_df.apply(
    lambda row: mode_country[row['response_corporate']] if pd.isnull(row['customer_country']) else row['customer_country'],
    axis=1
)

# product_category - 김주호
- product_subcategory,product_modelname을 이용해서 정리

In [233]:
#sub랑modelname에서 최대한 끌어오기
train_df.loc[train_df['product_subcategory'].isna(), 'product_subcategory'] = train_df['product_modelname']
train_df.loc[train_df['product_category'].isna(), 'product_category'] = train_df['product_subcategory']

In [234]:
#hotel tv
hotel_list=['hospitality','hoteleria_us670h','awz','bwz','酒店電視','43us660h']

#display랑 monitor랑 묶을 가능성 농후
medical_display_list=['medical','Medical Display','Clinical Review','Diagnostic ','X-ray','Monitor De Diagnostico','Surgical Monitors','radiology displays','surgical monitor','จอภาพสำหรับการตรวจสอบทางคลินิก','จอภาพเพื่อการวินิจฉัย','21HQ513D','31HN713D']
#그냥 monitor랑 display로 나뉜거랑 큰 차이없긴한데,
monitor_list=['Ergonomic monitors','IPS Monitors','Monitors for graphics applications & video editing','Monitors with USB-C connection','UHD 4K Monitors',
         '32UN880','34WN780','UltraWide™ Monitors','computer monitors','laec015','monior','Monitor','28MQ780','27QP88D']

#interactive digital board는 광범위긴함 원퀵도 포함될듯
interactive_list=['collaboration displays','互動式顯示屏','43HT3WJ','55CT5WJ','55tc3d']
#signage videowall,tv 묶을 가능성 농후   onequick도 가능성있음 + 어차피 다 interactive로 가도됨
#signage라는게 매우매우애매하다. 내생각엔 signage최소 signage끼리묶자.솔직히 videowall이랑 tv도 똑같긴함 edu는 모델명들이 signage쓰는거인듯 구글링결과  //videwall
video_wall_signage_list=['lg led bloc','signage','education createboard','fhd series','high brightness','LAD033F','led 顯示屏','ledallinone','lsca039','smart tv','ultra stretch series','ur640',
                         '標準顯示屏','特別顯示屏','高亮度顯示屏','49vl5f','49vl5g-m','55svh7f-a','55vm5e-a','55vm5j-h','65EP5G OLED Pro','led cinema','transparent oled','standard']
#tv로냅둘지 모니터로갈지 디스플레이로갈지
tv_list=['TV Monitors','comercial tv','commercial tv','led 70m2','taa lcd lfd displays','tv','醫院電視']
#lg one:quick series로갈것
onequick_list=['SC-00DA','aio','idb','one:quick','one quick','onequick']

#other또는 etc.으로갈것>>>other도 etc.으로 가자
etc_list=['AI/Machine Learning | Antennas, Transmitters and Towers | Audience Measurement | Cameras and Lenses',
            'Architect , We are Meeting for Enqiry Generation ( This is not a Inquiry)','Category','Channel Partner done FTS Details in System',
          'other','Thin Clients','Zero Clients','a definir','ahu','aircare','allinone_rmk','autre','corpouh5f','corpuh5f-','ctv','ems,audio/video','energy storage system','error',
          'essential series','gsca046','gscd046','gscd100','high inch 86 / 98 or 110','htv','id','inne','khác','lainnya','leadallin','lg customer care program','lg home bliss air solution',
          'lg magnit','lg paradise air solution','lg salang air solution for dream homes','meeting & screen sharedirect view leddirect view led','not specified','otros','outros','pantalla led outdoor',
          'parts','pro:centric','pro centric hotel','procentric','retaildigital','robot','sales inquiry','scroll compressor',
          'single package','solar','standalone','technical support','tr3','uh','unitario','vb.','ventilation','	window facing display','אחר','آخر','ฯลฯ','其他','軟體','laec15','43 pol','32 pol','led','window facing display']

#이후 vrf로묶을가능성 농후 (vrf,split,rac)
vrf_list=['all lg vrf systems','kimatyzacja vrf','vrf']
#multi-split
multi_list=['klimatyzacja multi-split','multi','פיצול מרובה']
#single-split
single_list=['single split','split tunggal']
chiller_list=['chiller','مبرد (تشيلر)']
#가정용에어컨쪽 sac의 subcategory는 vrf이고 vrf는 냉각쪽장치
rac_list=['ac rumah','aire acondicionado residencial','ar condicionado residencial','cac','climatiseur résidentiel','rac','residential air conditioner','sac','system ac','system air conditioner','teto ou cassete inverter','điều hòa cục bộ','điều hòa gia dụng','מזגנים למקום מגורים',
     'تكييف وتبريد','تكييفات','เครื่องปรับอากาศเผื่อที่อยู่อาศัย','on or several reversible ac']

#software solution
software_solution_list=['LG SuperSign CMS','solution']

cloud_device_list=['cloud']

projector_list=['project','ProBeam','bu50nst']

heating_list=['a thermodynamic water heater','aquecimento','calefacción','isıtma','חימום','حلول التدفئة','ogrzewanie']


#냉각기
refrigerator_list=['refrigerator','pendingin','soğutucu','systèmes de débit à réfrigérant variable (drv)']
washing_machine_list=['washing machine']

pc_list=['pc']
#'accessories'는 건들지말아보는게?

category_lists = [hotel_list,medical_display_list, monitor_list, etc_list, interactive_list,video_wall_signage_list, tv_list,onequick_list,vrf_list,multi_list,single_list,chiller_list,rac_list,software_solution_list,cloud_device_list,projector_list,heating_list,refrigerator_list,washing_machine_list,pc_list]
category_names = ['hotel tv','medical display', 'monitor', 'etc.', 'interactive digital board', 'video wall signage', 'tv', 'lg one:quick series','vrf','multi-split',"single-split",'chiller','rac','software solution','cloud device','projector','heating','refrigerator','washing_machine','pc']


for i in range(len(category_lists)):
    for item in category_lists[i]:
        train_df.loc[train_df['product_category'].str.contains(item, case=False, na=False, regex=False), 'product_category'] = category_names[i]

https://github.com/justgotothedesk/LGAimers/blob/main/CatBoost/CatBoost(20240214_%200.608826).ipynb

In [235]:
drop_col = ['customer_country.1']
train_df.drop(drop_col, axis=1, inplace=True)
test_df.drop(drop_col, axis=1, inplace=True)

In [236]:
label_columns = list(train_df.columns)

label_columns.remove("is_converted")

In [237]:
'''
# 수치형 변수 전체 Standardizaiton, Normalization 실행
'''
# Initialize scalers
scaler_standard = StandardScaler()
scaler_minmax = MinMaxScaler()
columns_to_ST = ["com_reg_ver_win_rate", "historical_existing_cnt","lead_desc_length"]
columns_to_NM = ["ver_win_rate_x", "ver_win_ratio_per_bu"]

# Apply Standardization to each column
for column in columns_to_ST:
    # Standardization
    train_df[column + '_standardized'] = scaler_standard.fit_transform(train_df[[column]])
    test_df[column + '_standardized'] = scaler_standard.fit_transform(test_df[[column]])
    # Drop the original column
    train_df.drop(columns=[column], inplace=True)
    test_df.drop(columns=[column], inplace=True)

# Apply Standardization to each column
for column in columns_to_NM:
    # Normalization
    train_df[column + '_normalized'] = scaler_minmax.fit_transform(train_df[[column]])
    test_df[column + '_normalized'] = scaler_minmax.fit_transform(test_df[[column]])
    # Drop the original column
    train_df.drop(columns=[column], inplace=True)
    test_df.drop(columns=[column], inplace=True)

# Print the first few rows of the DataFrame to check the results
print(train_df.head())

   bant_submit customer_country business_unit  customer_idx  \
0          1.0      Philippines            AS         32160   
1          1.0      Philippines            AS         23122   
2          1.0            India            AS          1755   
3          1.0            India            AS          4919   
4          1.0            India            AS         17126   

           customer_type  enterprise  id_strategic_ver  it_strategic_ver  \
0           End-Customer  Enterprise               NaN               NaN   
1           End-Customer  Enterprise               NaN               NaN   
2           End-Customer  Enterprise               NaN               NaN   
3           End-Customer  Enterprise               NaN               NaN   
4  Specifier/ Influencer  Enterprise               NaN               NaN   

   idit_strategic_ver             customer_job  ... ver_pro  \
0                 NaN                 purchase  ...       0   
1                 NaN  media and commu

In [240]:
'''
# business_area 날리고 2개 피처 추가
'''
def is_hospital(value):
    if value == "hospital & health care":
        return 1
    else:
        return 0

def is_power(value):
    if value == "power plant / renewable energy":
        return 1
    else:
        return 0

# "business_area" 열에서 함수를 적용하여 새로운 열 생성
train_df['is_hospital'] = train_df['business_area'].apply(is_hospital)
train_df['is_power'] = train_df['business_area'].apply(is_hospital)
test_df['is_hospital'] = test_df['business_area'].apply(is_hospital)
test_df['is_power'] = test_df['business_area'].apply(is_hospital)

train_df.drop(columns=["business_area"], inplace=True)
test_df.drop(columns=["business_area"], inplace=True)

In [242]:
'''
# expected_timeline 에서 특정 단어 필터링
'''
# 새로운 열을 추가하기 위한 함수 정의
def contains_budget(value):
    if 'budget' in str(value).lower():  # 대소문자 구분없이 'budget'이 포함되어 있는지 확인
        return 1
    else:
        return 0

# "expected_timeline" 열에서 함수를 적용하여 새로운 열 생성
train_df['contains_budget'] = train_df['expected_timeline'].apply(contains_budget)
test_df['contains_budget'] = test_df['expected_timeline'].apply(contains_budget)

# 새로운 열을 추가하기 위한 함수 정의
def contains_etc(value):
    if 'etc' in str(value).lower():  # 대소문자 구분없이 'etc'이 포함되어 있는지 확인
        return 1
    else:
        return 0

# "expected_timeline" 열에서 함수를 적용하여 새로운 열 생성
train_df['contains_etc'] = train_df['expected_timeline'].apply(contains_etc)
test_df['contains_etc'] = test_df['expected_timeline'].apply(contains_etc)

# 새로운 열을 추가하기 위한 함수 정의
def contains_hence(value):
    if 'hence' in str(value).lower():  # 대소문자 구분없이 'hence'이 포함되어 있는지 확인
        return 1
    else:
        return 0

# "expected_timeline" 열에서 함수를 적용하여 새로운 열 생성
train_df['contains_hence'] = train_df['expected_timeline'].apply(contains_hence)
test_df['contains_hence'] = test_df['expected_timeline'].apply(contains_hence)

# 새로운 열을 추가하기 위한 함수 정의
def contains_system(value):
    if 'system' in str(value).lower():  # 대소문자 구분없이 'system'이 포함되어 있는지 확인
        return 1
    else:
        return 0

# "expected_timeline" 열에서 함수를 적용하여 새로운 열 생성
train_df['contains_system'] = train_df['expected_timeline'].apply(contains_system)
test_df['contains_system'] = test_df['expected_timeline'].apply(contains_system)

# 새로운 열을 추가하기 위한 함수 정의
def contains_closi(value):
    if 'closi' in str(value).lower():  # 대소문자 구분없이 'closi'이 포함되어 있는지 확인
        return 1
    else:
        return 0

# "expected_timeline" 열에서 함수를 적용하여 새로운 열 생성
train_df['contains_closi'] = train_df['expected_timeline'].apply(contains_closi)
test_df['contains_closi'] = test_df['expected_timeline'].apply(contains_closi)

# 새로운 열을 추가하기 위한 함수 정의
def contains_any(value):
    if 'any' in str(value).lower():  # 대소문자 구분없이 'any'이 포함되어 있는지 확인
        return 1
    else:
        return 0

# "expected_timeline" 열에서 함수를 적용하여 새로운 열 생성
train_df['contains_any'] = train_df['expected_timeline'].apply(contains_any)
test_df['contains_any'] = test_df['expected_timeline'].apply(contains_any)

# 새로운 열을 추가하기 위한 함수 정의
def contains_although(value):
    if 'although' in str(value).lower():  # 대소문자 구분없이 'although'이 포함되어 있는지 확인
        return 1
    else:
        return 0

# "expected_timeline" 열에서 함수를 적용하여 새로운 열 생성
train_df['contains_although'] = train_df['expected_timeline'].apply(contains_although)
test_df['contains_although'] = test_df['expected_timeline'].apply(contains_although)

# 새로운 열을 추가하기 위한 함수 정의
def contains_more(value):
    if 'more' in str(value).lower():  # 대소문자 구분없이 'more'이 포함되어 있는지 확인
        return 1
    else:
        return 0

# "expected_timeline" 열에서 함수를 적용하여 새로운 열 생성
train_df['contains_more'] = train_df['expected_timeline'].apply(contains_more)
test_df['contains_more'] = test_df['expected_timeline'].apply(contains_more)

# 새로운 열을 추가하기 위한 함수 정의
def contains_year(value):
    if 'year' in str(value).lower():  # 대소문자 구분없이 'year'이 포함되어 있는지 확인
        return 1
    else:
        return 0

# "expected_timeline" 열에서 함수를 적용하여 새로운 열 생성
train_df['contains_year'] = train_df['expected_timeline'].apply(contains_year)
test_df['contains_year'] = test_df['expected_timeline'].apply(contains_year)

In [243]:
x_train, x_val, y_train, y_val = train_test_split(
    train_df.drop("is_converted", axis=1),
    train_df["is_converted"],
    test_size=0.2,
    shuffle=True,
    random_state=400,
)

In [244]:
cat_features_col = list(x_train.columns)
cat_features_col.remove("bant_submit")
cat_features_col.remove("com_reg_ver_win_rate_standardized")
cat_features_col.remove("historical_existing_cnt_standardized")
cat_features_col.remove("id_strategic_ver")
cat_features_col.remove("it_strategic_ver")
cat_features_col.remove("idit_strategic_ver")
cat_features_col.remove("lead_desc_length_standardized")
cat_features_col.remove("ver_cus")
cat_features_col.remove("ver_pro")
cat_features_col.remove("ver_win_rate_x_normalized")
cat_features_col.remove("ver_win_ratio_per_bu_normalized")

In [245]:
# Calculate the most frequent value (mode) for each column
modes = x_train.mode().iloc[0]

# Fill missing (NA) values with the mode
x_train.fillna(modes, inplace=True)

catboost_model = CatBoostClassifier(
    verbose=0,
    iterations=1000,
    depth=6,
    learning_rate=0.1,
    loss_function='Logloss',
    one_hot_max_size=5,
    cat_features=cat_features_col
    )

In [246]:
catboost_model.fit(x_train, y_train)

In [247]:
# 모델을 사용하여 검증 데이터셋에 대한 예측 수행
# Calculate the most frequent value (mode) for each column
modes = x_val.mode().iloc[0]

# Fill missing (NA) values with the mode
x_val.fillna(modes, inplace=True)

y_val_pred = catboost_model.predict(x_val)

y_val_pred = np.array([True if pred == 'True' else False for pred in y_val_pred])

print("Classification Report:\n", classification_report(y_val, y_val_pred))
print("오차행렬:\n", confusion_matrix(y_val, y_val_pred, labels=[True, False]))
print("\n정확도: {:.4f}".format(accuracy_score(y_val, y_val_pred)))
print("정밀도: {:.4f}".format(precision_score(y_val, y_val_pred, labels=[True, False])))
print("재현율: {:.4f}".format(recall_score(y_val, y_val_pred)))
print("F1: {:.4f}".format(f1_score(y_val, y_val_pred, labels=[True, False])))

Classification Report:
               precision    recall  f1-score   support

       False       0.98      1.00      0.99     10913
        True       0.94      0.76      0.84       947

    accuracy                           0.98     11860
   macro avg       0.96      0.88      0.91     11860
weighted avg       0.98      0.98      0.98     11860

오차행렬:
 [[  720   227]
 [   47 10866]]

정확도: 0.9769
정밀도: 0.9387
재현율: 0.7603
F1: 0.8401


In [249]:
x_test = test_df.drop(["is_converted", "id"], axis=1)

# Calculate the most frequent value (mode) for each column
modes = x_val.mode().iloc[0]

# Fill missing (NA) values with the mode
x_val.fillna(modes, inplace=True)

test_pred = catboost_model.predict(x_test)

df_sub = pd.read_csv("/content/drive/MyDrive/lg/submission.csv")
df_sub["is_converted"] = test_pred

# 제출 파일 저장
df_sub.to_csv("/content/drive/MyDrive/lg/submissioncat0215.csv", index=False)

CatBoostError: Invalid type for cat_feature[non-default value idx=3,feature_idx=4]=nan : cat_features must be integer or string, real number values and NaN values should be converted to string.